Futures Bull Vertical Screener
=======

We want to find high ROC credit bull PUT verticals for future contracts, in 'underlying_symbol' underlyings that comply with :

* future contracts for months forward up to 'months_forward'
* Average option volume > 'min_option_volume'.
* Iv rank > min_iv_rank

After previous filter, scan option chains for remaining underlyings:

* Explore up to 'num_month_expiries' monthly option expiries forward from current date.
* Scan combinations of strike prices starting from smallest strike to  (ATM - 'pct_under_px_range'% ) range. Zero explores all available strike combinations.
* Limit risk to 'max_risk' USD (which influences allowable distance between strikes).

Notes:
1. If notional value of a naked put strategy is less than 'max_risk' then switch to naked put strategy (do not buy 2long leg).
2. Order resulting strategies descending on 'Return on Capital' (MaxProfit / Margin ).
3. If 'ask_tws_load==True', user will be prompted to load strategies on TWS.
4. Only OTM options will be scaned (strikes below current price)

In [81]:
from ib_insync import *

from datetime import datetime, timezone, timedelta

import pandas as pd


In [82]:
underlying_symbol='CL'

#underlying filters
months_forward = 1
min_option_open_interest = 100000
min_iv_rank = .10 #10% = .10

#option strategy filters
pct_under_px_range = .20 # scan strikes under this % below cunderlying price 
num_month_expiries = 3 #expires forward
max_loss = 2000 # max loss
min_profit = 200 #min profit
ask_tws_load =True #ask wheter to load strategies

In [83]:
util.startLoop()

ib = IB()

if ib.isConnected() == False:
    ib.connect('127.0.0.1', 7497, clientId=18, readonly = True)

<IB connected to 127.0.0.1:7497 clientId=18>

### Initial filtering

Search future contracts on 'underlying_symbol' commodity with filters: 'months_forward', 'min_option_volume', 'min_iv_rank'

In [84]:
#get 'months_forward' month expiration datetime
curdate = datetime.now()
three_months = timedelta( weeks=months_forward*4 )
expiration_limit = curdate + three_months

expiration_limit

datetime.datetime(2020, 7, 4, 21, 59, 19, 681301)

In [85]:
contracts = []

contract = Future(symbol=underlying_symbol, exchange='NYMEX', currency='USD')

contracts = ib.reqContractDetails(contract)

print('{} contracts found related to {}'.format( len(contracts), underlying_symbol ) )

128 contracts found related to CL


**Filter by months_forward**

In [86]:
filteredContracts = []

for c in contracts:
    contract_exp = pd.to_datetime( c.contract.lastTradeDateOrContractMonth,  format='%Y%m%d' )
    if contract_exp < expiration_limit:
        filteredContracts.append(c.contract)

print( '{} contracts after months_forward = {} filter'.format( len( filteredContracts ), months_forward ) )

filteredContracts

1 contracts after months_forward = 1 filter


[Contract(secType='FUT', conId=174230636, symbol='CL', lastTradeDateOrContractMonth='20200622', multiplier='1000', exchange='NYMEX', currency='USD', localSymbol='CLN0', tradingClass='CL')]

**Filter by minimum open interest**

This needs market data subscription.

In [87]:
tickers = {}

#generickTickList:
# 101: call option open interest, put option open interest
# 105: avOptionVolume (not used, not working)
# 106: impliedvolatility

#TODO: this might fail for larger contract lists 
for c in filteredContracts:
    tickers.update( { c.conId : ib.reqMktData( c, genericTickList='101,105,106' ) } )

#wait a couple of seconds for ticker info to be filled
ib.sleep( 2 )

True

Get sum call + put open interest and filter

In [88]:
for c in filteredContracts:
    ticker = tickers[ c.conId ]
    if (ticker.putOpenInterest + ticker.callOpenInterest) < min_option_open_interest:
        filteredContracts.remove( ticker.contract )
        ib.cancelMktData( ticker.contract )
        tickers.remove( c.conId )
        
print( '{} contracts after min_option_open_interest = {} '.format( len(filteredContracts), min_option_open_interest ) )

1 contracts after min_option_open_interest = 100000 


**Filter by IV rank**

In [89]:
#get IV historical bars

histBars = {}

for c in filteredContracts:
    bars = ib.reqHistoricalData(
            c,
            endDateTime='',
            durationStr='256 D',
            barSizeSetting='1 day',
            whatToShow='OPTION_IMPLIED_VOLATILITY',
            useRTH=True,
            formatDate=1)
    
    histBars.update( { c.conId : bars} )
    
ib.sleep(2)


True

In [90]:
#highest 53 W IV
ivHighs = {}
#current IV
ivCurrents = {}
#curent 52 W IV rank per contract
ivRanks = {}

#get highest IV / IV rank
for c in filteredContracts:
    curHistBars = histBars[ c.conId ]
    sortedBars = sorted( curHistBars, key=lambda bar: bar.close, reverse=True )
    ivHighs.update( { c.conId : sortedBars[0].close } )
    ivCurrents.update( { c.conId : curHistBars[-1].close } )
    print( 'IV rank for {} contract: {}'.format( c.conId, curHistBars[-1].close / sortedBars[0].close ) )
    ivRanks.update( { c.conId : curHistBars[-1].close / sortedBars[0].close } )
    


IV rank for 174230636 contract: 0.1886891883004968


In [91]:
for c in filteredContracts:
    if ivRanks[ c.conId ] < min_iv_rank:
        filteredContracts.remove(c)
        tickers.remove( c.conId )
        ib.cancelMktData( c )
        
        
print( '{} contracts surpased iv rank minumum if {}'.format(len(filteredContracts), min_iv_rank ) )
            

1 contracts surpased iv rank minumum if 0.1


**At this point screener has discriminated future contracts by:**

    * Expiration.
    * Open Interest (Put+Call Option contracts).
    * 52 week IV rank.

**Request option chains for remaining contracts**

In [92]:
chains = {}

for c in filteredContracts:
    chains.update( { c.conId : ib.reqSecDefOptParams(c.symbol, c.exchange, c.secType, c.conId ) } )

print('{} chains found'.format( len(chains) ) )

#util.df(chains)

1 chains found


From this point on we can build all PUT  option contracts that meet our conditions (Naked puts / Bull Put Spreads).

Detail of implemented algorithm:
  
1. Iterate over stock option 'chains', select first chain.
  1. filter the expirations list, limit to up to 3 month expirations.
  2. filter strike list, limit to strikes below 'pct_under_px_range'
  3. Explore remaining strike / expiriation combinations in filtered option chains:
     1. **Chains / Expirations loop**. Iterate over chains/ expirations, select first chain/expiration (might be only one per chain):
        1.  **Leg 1 loop**. Iterate over strikes, select leg 1 strike.
            1. Combine selected strike with current expiration from outer loop.
            2. Build Option contract.
            3. Request market data (price) for 1st leg ontract.
            4. Fix leg 1 contract for going to inner loop.
                 1. **Leg 2 loop**. Iterate over strikes, select leg 2 strike
                     1. Is strike < combo leg 1 strike? No : Discard / iterate again, Yes: Select it.
                     2. Combine selected strike with current expiration from outer loop
                     3. Build Option contract.
                     4. Request market data (price) for 2nd leg contract.
                     5. Calculate COMBO price (max profit) using 1st and 2nd leg market data.
                     6. Is maximum profit > 'min_profit' from screener parameters? No: Discard.
                     7. Calculate maximum loss using 1st and 2nd leg strike distance.
                     8. Is maximum loss > 'max_risk' from screener parameters? Yes: Discard.
                     9. Create 'COMBO' order,  request 'whatif' for price / margin information
                    10. Save 'COMBO' order / legs / ROC etc in local **results** data frame.
            5. Continue loop to step 1 in **Leg 1 loop**.
        2. Save local **results** array into **combosByChain** dictionary.
        3. Continue loop to step 1 in **Chains / Expirations loop**.
  4. Results are stored in 'combosByContract' dictionary.

In [93]:
# prepare 'num_month_expiries' datetime object
curdate = datetime.now()
delta_forward = timedelta( weeks = num_month_expiries*4 )
option_expiration_limit = curdate + delta_forward

option_expiration_limit

datetime.datetime(2020, 8, 29, 21, 59, 24, 956157)

## Filter option chains

Filter option chains for filtered contracts according to:
* **'pct_under_px_range'** (percentage below price)  
* **'option_expiration_limit'** (option expiration)

In [94]:
# filter chains strikes / expirations:

adjustedContractChains = {}

for c in filteredContracts:
    adjustedChains = []
    curChains = chains[ c.conId  ]
    curPrice = tickers[ c.conId ].marketPrice() # might need to replace by 'saved' value
    print( 'current price for contract {} : {}'.format( c.conId, curPrice ) )
    
    for chain in curChains:
        
        expdate = pd.to_datetime( chain.expirations[ 0 ]  )
        
        # expiration within range? (suppose sorted already)
        if( expdate < option_expiration_limit ):
            #filter strikes ('pct_under_px_range')
            newstrikes = [ s for s in chain.strikes if s > ( curPrice * ( 1 - pct_under_px_range ) ) and s <= curPrice ]
                        
            #OptionChain is 'NamedTuple' (not mutable)
            adjustedChain = OptionChain( chain.exchange, 
                                        chain.underlyingConId,
                                        chain.tradingClass,
                                        chain.multiplier,
                                        chain.expirations,
                                        newstrikes)
            #adjustedChain.strikes = newstrikes
            adjustedChains.append( adjustedChain )
    
    adjustedContractChains.update( { c.conId : adjustedChains } )

current price for contract 174230636 : 37.41


### Combo screening

Create L1/L2 combo orders, calculate prices, filter by min_profit, max_loss, save into **combosByContract** / **combosByChain** dictionaries


In [95]:
# screened contracts will be placed here, one list per contract
# { conId : { combo : orderState } }
combosByContract = {}
combosByChain = {}


# Create / Filter combos
for c in filteredContracts:

    for chain in adjustedContractChains[ c.conId ]:
            
            results = []

            #chainCombos = pd.DataFrame( columns=cols )
            
            #suppose only 1 expiration per chain
            comboContracts = [FuturesOption(c.symbol, chain.expirations[0], strike, 'P', c.exchange )
                    for strike in chain.strikes]

            comboContracts = ib.qualifyContracts( *comboContracts )
            
            # should be less than 100 requests (request limit)
            comboTickers = ib.reqTickers( *comboContracts )
                        
            print( '{} tickers received for combo {} contract chain'.format( len(comboTickers), chain.underlyingConId ) )
            
            # ---- leg1 loop ----
            for leg1Ticker in comboTickers:
                leg1Price = leg1Ticker.marketPrice()
                leg1Strike = leg1Ticker.contract.strike
                 # early filter leg1 by min_profit
                if( ( leg1Price * int(chain.multiplier) ) < min_profit ):
                    comboContracts.remove(leg1Ticker.contract)
                else:
                    #fix leg 1
                    leg1 = ComboLeg(conId=leg1Ticker.contract.conId, 
                                    ratio= 1, 
                                    action='SELL', 
                                    exchange=leg1Ticker.contract.exchange )
                    # ---- leg 2 loop ----
                    for leg2Ticker in comboTickers:
                        leg2Strike = leg2Ticker.contract.strike
                        if( leg2Ticker.contract.strike < leg1Strike ):
                            leg2Price = leg2Ticker.marketPrice()
                            #leg2 hedge should cost less than leg1
                            comboPrice = ( leg1Price - leg2Price ) * int(chain.multiplier)
                            
                            if ( comboPrice < max_loss ) and \
                            ( comboPrice > min_profit ):
                                    
                                leg2 = ComboLeg( conId=leg2Ticker.contract.conId,  
                                                ratio=1,
                                                action='BUY',
                                                exchange=leg2Ticker.contract.exchange )

                                #create 'combo contract'
                                combo = Contract( symbol=leg1Ticker.contract.symbol,#check!! 
                                                 secType='BAG', 
                                                 currency='USD', 
                                                 exchange=leg1Ticker.contract.exchange, 
                                                 comboLegs=[leg1, leg2])

                                order = MarketOrder( action="BUY", 
                                                   totalQuantity = 1 )

                                #this method is blocking
                                orderState = ib.whatIfOrder( combo, order )
                                
                                roc = comboPrice / float(orderState.maintMarginChange)

                                #model greeks might not be pupulated (check)
                                if leg1Ticker.modelGreeks != None :
                                    leg1IV = leg1Ticker.modelGreeks.impliedVol
                                    leg2IV = leg2Ticker.modelGreeks.impliedVol
                                else:
                                    leg1IV = float('nan')
                                    leg2IV = float('nan')

                                    

                                results.append( [ leg1Strike,
                                                   leg1Ticker.contract.conId,
                                                   leg1IV,
                                                   leg2Strike,
                                                   leg2Ticker.contract.conId,
                                                   leg2IV,
                                                   comboPrice,
                                                   float(orderState.maintMarginChange),
                                                   roc ] )
                                
            
            combos_df = pd.DataFrame( data = results, columns=[ 'leg1Strike', 
                                                                'leg1ContractId',
                                                                'leg1IV',
                                                                'leg2Strike',
                                                                'leg2ContractId',
                                                                'leg2IV',
                                                                'maxProfit',
                                                                'margin',
                                                                'ROC' ] )
            
            print( '{} combos found for chain with trading class {} and expiration {}'.format( \
                len( combos_df.index ), chain.tradingClass, chain.expirations[0] ) )
            
            combosByChain.update( { chain.expirations[0] : combos_df } )
                        
    combosByContract.update( { c.conId: combosByChain } )

15 tickers received for combo 174230636 contract chain
64 combos found for chain with trading class LO2 and expiration 20200612
15 tickers received for combo 174230636 contract chain
70 combos found for chain with trading class LO and expiration 20200617


### Sort ascending by 'ROC' column

In [103]:

for ( expiration, chain ) in combosByChain.items():
    chain.sort_values( by='ROC', axis=0, ascending=False, inplace=True, ignore_index=True )

## RESULTS

At this point we should have:

- **combosByContract** : dictionary, all combos per expiration grouped by contracts
- **combosByChain** : dictionary, all combos grouped by expiration
- **IvRanks**: dictionary, ivRanks for each of the possible underlying contracts

It should be straightforward to analyze and visually represent the best strategies.


In [11]:
#ib.disconnect()